# Sentence correction gpt on Korean

한국어 문법교정 GPT3 모델 구축하기


In [3]:
%pip install pandas numpy scipy scikit-learn torch accelerate transformers ipywidgets tqdm

# Transformers 를 사용하기 위한 라이브러리들 설치

  Using cached pandas-2.2.1-cp312-cp312-macosx_11_0_arm64.whl.metadata (19 kB)
  Using cached numpy-1.26.4-cp312-cp312-macosx_11_0_arm64.whl.metadata (61 kB)
  Using cached scipy-1.12.0-cp312-cp312-macosx_12_0_arm64.whl.metadata (217 kB)
  Using cached scikit_learn-1.4.1.post1-cp312-cp312-macosx_12_0_arm64.whl.metadata (11 kB)
  Using cached torch-2.2.1-cp312-none-macosx_11_0_arm64.whl.metadata (25 kB)
  Using cached accelerate-0.28.0-py3-none-any.whl.metadata (18 kB)
  Using cached transformers-4.39.0-py3-none-any.whl.metadata (134 kB)
  Using cached ipywidgets-8.1.2-py3-none-any.whl.metadata (2.4 kB)
  Using cached tqdm-4.66.2-py3-none-any.whl.metadata (57 kB)
  Using cached pytz-2024.1-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2024.1-py2.py3-none-any.whl.metadata (1.4 kB)
  Using cached joblib-1.3.2-py3-none-any.whl.metadata (5.4 kB)
  Using cached threadpoolctl-3.4.0-py3-none-any.whl.metadata (13 kB)
  Using cached filelock-3.13.1-py3-none-any.whl.metadata (2.8 kB

### AI-Hub data 정제

예시 데이터는 다음과 같다.

이 프로젝트를 수행하는 데이터인 AI-Hub 데이터는 용량이 112GB, 국외반출 불가로 데이터 관리에 주의를 요한다.

이 데이터를 Pandas.DataFrame['index', text', 'corrected'] 로 정제하는 과정을 거친다.

```json
{  "id": "100008-1-1-1",
  "fileName": "TX_CA_1_100008-1-1-1",
  "dataSet": "한국어 철자 및 맞춤법 교정용 병렬 데이터",
  "domain": "CA",
  "ko": "지금까지 다녀 본 여행지 중 좋았던 곳 추천해줘.",
  "corrected": "지금까지 다녀 본 여행지 중 좋았던 곳 추천해 줘.",
  "error": [
    {
      "errorType": "spac",
      "startPoint": 22,
      "endPoint": 27
    }
  ]
}
```

In [ ]:
from typing import List, Dict
from tqdm.notebook import tqdm

import json
import pandas as pd

LOCATION = '.'

def read_json_file(file_path: str) -> pd.DataFrame:
    """
    JSON 파일을 읽어와 파싱한 데이터를 리스트로 반환하는 함수
    :param file_path: 읽어올 JSON 파일의 경로
    :return: JSON 파일을 파싱한 리스트
    """

    df = pd.DataFrame(columns=['text', 'corrected'])

    # JSON 파일 읽어오기
    with open(file_path, 'r') as f:
        data: List[Dict] = json.load(f)

        for p in tqdm(data):
            df.loc[len(df)] = {'text': p.get('ko'), 'corrected': p.get('corrected')}

train = [

]
validate = [

]

train_data = pd.DataFrame()
validate_data = pd.DataFrame()

for file in tqdm(train):
    train_data = pd.concat([train_data, read_json_file(f'{LOCATION}/{file}')], ignore_index=True)
for file in tqdm(validate):
    validate_data = pd.concat([validate_data, read_json_file(f'{LOCATION}/{file}')], ignore_index=True)

train_data.to_csv(f'{LOCATION}/train.csv')
validate_data.to_csv(f'{LOCATION}/validate.csv')

### Transformer 학습 준비

학습 준비된 데이터는 토크나이저를 불러와 DataLoader에 준비하여 학습 준비를 한다.

파인튜닝 대상 모델은 다음과 같다.
[kykim/gpt3-kor-small_based_on_gpt2](https://github.com/kiyoungkim1/LMkor)

In [ ]:
from torch.utils.data import Dataset, DataLoader
from transformers import GPT2LMHeadModel, GPT2Tokenizer

import pandas as pd
import torch

LOCATION = '.'

# GPT-3 모델과 토크나이저 불러오기
tokenizer = GPT2Tokenizer.from_pretrained('kykim/gpt3-kor-small_based_on_gpt2')
max_length = 100
loader_size = 16

# 훈련용 및 검증용 데이터셋 클래스 정의
class TextDataset(Dataset):
    def __init__(self, data, tokenizer, max_length):
        self.data = data
        self.tokenizer = tokenizer
        self.max_length = max_length
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        input_text = self.data.iloc[idx]['text']
        output_text = self.data.iloc[idx]['corrected']
        input_ids = self.tokenizer.encode(input_text, max_length=self.max_length, truncation=True, padding='max_length')
        output_ids = self.tokenizer.encode(output_text, max_length=self.max_length, truncation=True, padding='max_length')
        return {
            'input_ids': torch.tensor(input_ids, dtype=torch.long),
            'output_ids': torch.tensor(output_ids, dtype=torch.long)
        }

train = pd.read_csv(f'{LOCATION}/train.csv')
validate = pd.read_csv(f'{LOCATION}/validate.csv')

train_dataset = TextDataset(train, tokenizer, max_length)
validate_dataset = TextDataset(validate, tokenizer, max_length)

train_loader = DataLoader(train_dataset, batch_size=loader_size, shuffle=True)
validate_loader = DataLoader(validate_dataset, batch_size=loader_size, shuffle=True)

In [ ]:
from torch.utils.data import Dataset, DataLoader
from transformers import GPT2LMHeadModel, GPT2Tokenizer
from tqdm.notebook import tqdm

import pandas as pd
import torch

# 모델 및 옵티마이저 불러오기
model = GPT2LMHeadModel.from_pretrained('kykim/gpt3-kor-small_based_on_gpt2')
optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)

# 학습 및 검증 함수 정의
def train(train_loader, model, optimizer):
    model.train()
    total_loss = 0.0
    for batch in tqdm(train_loader):
        inputs = batch['input_ids'].to(model.device)
        labels = batch['output_ids'].to(model.device)
        optimizer.zero_grad()
        outputs = model(inputs, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(train_loader)

def validate(validate_loader, model):
    model.eval()
    total_loss = 0.0
    with torch.no_grad():
        for batch in tqdm(validate_loader):
            inputs = batch['input_ids'].to(model.device)
            labels = batch['output_ids'].to(model.device)
            outputs = model(inputs, labels=labels)
            loss = outputs.loss
            total_loss += loss.item()
    return total_loss / len(validate_loader)

# 학습 및 검증 수행
epochs = 3
for epoch in tqdm(range(epochs)):
    train_loss = train(train_loader, model, optimizer)
    validate_loss = validate(validate_loader, model)
    print(f"Epoch {epoch + 1}/{epochs}, Train Loss: {train_loss}, Validate Loss: {validate_loss}")